In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import gc
import warnings
warnings.simplefilter('ignore')

In [5]:
from sklearn.preprocessing import LabelEncoder

In [37]:
train_ = pd.read_csv('Train.csv')
test_ = pd.read_csv('Test.csv')
submission_ = pd.read_csv('SampleSubmission.csv')

In [38]:
train_['sum'] = train_[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']].T.sum()

test_['sum'] = test_[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']].T.sum()+1

In [39]:
train_= train_[['sum','ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3',
       ]]
test_= test_[['sum','ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3',
       ]]

In [42]:
from sklearn.model_selection import KFold

def get_train_test_names(train_, test_, submission_):
  kf = KFold(n_splits=5, shuffle=False)
  for r, (train_index, test_index) in enumerate(kf.split(train_)):
    test = train_.iloc[test_index]

    X_test = []
    X_test_columns = test.columns
    for v in test.values:
      info = v[:9]
      binary = v[9:]
      index = [k for k, i in enumerate(binary) if i == 1]
      for i in index:
        for k in range(len(binary)):
          if k == i:
            binary_transformed = list(copy.copy(binary))
            binary_transformed[i] = 0
            X_test.append(list(info) + binary_transformed)

    X_test = pd.DataFrame(X_test)
    X_test.columns = ['sum','ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
          'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
          '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
          'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']

    X_test['ID'] = [str(r)+'_'+str(i) for i in range(X_test.shape[0])]

    yield train_.iloc[train_index], X_test, submission_, '2_fold' + str(r) + '.csv'
  yield train_, test_, submission_, '2_main.csv'

In [43]:
def process(df):
  
  
  df['eclapse'] = 2020 - df['date3']
  df['IS_748L'] = df['branch_code'] == '748L'
  df['IS_748L'] = df['IS_748L'].astype('int')
  
  df['IS_T4MS'] = df['occupation_category_code'] == 'T4MS'
  df['IS_T4MS'] = df['IS_T4MS'].astype('int')
  
  df['IS_RVSZ'] = df['PCODE'].apply(lambda x : 1 if x=='RVSZ' else 0 )
  df['IS_K6QO'] = df['PCODE'].apply(lambda x : 1 if x=='K6QO' else 0 )

  df['IS_RVSZ'] = df['PCODE'].apply(lambda x : 1 if x=='RVSZ' else 0 )
  df['IS_K6QO'] = df['PCODE'].apply(lambda x : 1 if x=='K6QO' else 0 )
  df['IS_M'] = df['marital_status'].apply(lambda x : 1 if x=='M' else 0 )
  df['current_age'] = np.abs(2020 - df['birth_year'])
  


  return df

In [44]:
for train, test, submission, name in get_train_test_names(train_, test_, submission_):
  print(40* '=')
  i = 1 
  print(f'{i} / 5')
  true_values = []
  for v in test.values:
    binary = v[9:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in test.columns[9:][index]:
      true_values.append(v[1] + ' X ' + k)

  #Store Products ID in Multidimensional array
  products =train[['P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
  products_t =test[['P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]

  #Axis Transformation
  train = train.melt(id_vars=train.columns[:9], value_vars=products, var_name = "PCODE", value_name="Label" )
  test = test.melt(id_vars=test.columns[:9], value_vars=products, var_name = "PCODE", value_name="Label" )

  #A merger
  train['combiner']='x'

  #Set
  data=pd.concat([train,test],sort=False).reset_index(drop=True)

  train=data[data.combiner.notnull()].reset_index(drop=True);test=data[data.combiner.isna()].reset_index(drop=True)
  train.drop('combiner', inplace=True, axis=1);test.drop(['Label','combiner'], inplace=True, axis=1)

  #Concatenate new column towards submission requirement
  train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE'];test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

  #Re-arrange column (not necessary)
  train=train[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE','sum',  'Label']]
  test=test[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE','sum',]]


  train['date1'] = train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
  train['date2'] = train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
  train['date3'] = train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)

  test['date1'] = test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
  test['date2'] = test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
  test['date3'] = test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)

  ## 1 - join date feature engineering 
  train.join_date = pd.to_datetime(train.join_date, 
                                  #format="%Y/%m%d"
                                  )
  test.join_date = pd.to_datetime(test.join_date,
                                  #format="%Y-%m-%d"
  )
  train.join_date = pd.to_datetime(train.join_date, format="%Y/%m%d")
  test.join_date = pd.to_datetime(test.join_date, format="%Y-%m-%d")

  train['day_of_week'] = train['join_date'].dt.dayofweek
  train['day_of_week_name'] = train['join_date'].dt.day_name()
  train['age'] = np.abs(train['join_date'].dt.year - train['birth_year'])

  test['day_of_week'] = test['join_date'].dt.dayofweek
  test['day_of_week_name'] = test['join_date'].dt.day_name()
  test['age'] = np.abs( test['join_date'].dt.year - test['birth_year'])

  for col in ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',
            'day_of_week_name','join_date'] :
        train[col] = train[col].astype('object')
        test[col] = test[col].astype('object')

  print('Process Train & Test')
  train = process(train)
  test = process(test)
  train['sum']=train['sum'].astype('int64')
  test['sum']=test['sum'].astype('int64')
  print('---------------------------------------------------------------------------------------------')
  print('train shape : ' , train.shape,'test shape : ' , test.shape)
  #train shape :  (611772, 24) test shape :  (210000, 23)
  print('---------------------------------------------------------------------------------------------')
  
  #########################################################################################################
  param = {
    'bagging_freq': 1,
    'bagging_fraction': 0.8,
    'feature_fraction': 0.9,
    'boost_from_average':'false',
    'num_leaves': 80,
    'boost': 'gbdt',
    'learning_rate': 0.05,
    'metric':'auc',
    'tree_learner': 'serial',
    'objective': 'binary',
    'random_state': 869,
    'n_jobs': -1,
    'verbosity': -1
  }

  fts = [x for x in train.columns if x not in ['IS_748L', 'IS_T4MS', 'IS_33_34_to_38',
       'IS_1993_1982_1984_and_T4MS', 'IS_month_4_5_and_T4MS'] +['ID', 'Label', 'ID X PCODE',] ]

  from sklearn import preprocessing


  for f in ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code','PCODE',
            'day_of_week_name','join_date']:
      
          lbl = preprocessing.LabelEncoder()
          lbl.fit(list(train[f].values) + list(test[f].values))
          train[f] = lbl.transform(list(train[f].values))
          test[f] = lbl.transform(list(test[f].values))   
  import lightgbm as lgb

  trn_data = lgb.Dataset(train[fts], label=train['Label'] ,
                         categorical_feature=['branch_code','occupation_code', 'occupation_category_code'])  
  
  lgb_model = lgb.train(param, trn_data, 1500,)
  print('Start Predicting....')
  proba = lgb_model.predict(test[fts],num_iteration=lgb_model.best_iteration)

  df_answer = test[['ID X PCODE']]
  df_answer['Label'] = proba

  df_answer.columns = ['ID X PCODE', 'Label']
  for i in range(df_answer.shape[0]):
    if df_answer['ID X PCODE'].iloc[i] in true_values:
      df_answer['Label'].iloc[i] = 1.0

  df_answer.reset_index(drop=True, inplace=True)
  df_answer.to_csv(name, index=False)
  i+=1

1 / 5
Process Train & Test
---------------------------------------------------------------------------------------------
train shape :  (489405, 25) test shape :  (279426, 24)
---------------------------------------------------------------------------------------------
Start Predicting....
1 / 5
Process Train & Test
---------------------------------------------------------------------------------------------
train shape :  (489405, 25) test shape :  (277977, 24)
---------------------------------------------------------------------------------------------
Start Predicting....
1 / 5
Process Train & Test
---------------------------------------------------------------------------------------------
train shape :  (489426, 25) test shape :  (276318, 24)
---------------------------------------------------------------------------------------------
Start Predicting....
1 / 5
Process Train & Test
---------------------------------------------------------------------------------------------
train 

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
!mkdir '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3'

In [47]:
!cp 2_fold0.csv '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3'
!cp 2_fold1.csv '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3'
!cp 2_fold2.csv '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3'
!cp 2_fold3.csv '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3'
!cp 2_fold4.csv '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3'
!cp 2_main.csv '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3'